In [1]:
import torch

In [2]:
a = 1e-12

In [3]:
type(a)

float

In [5]:
torch.tensor(a)

tensor(1.0000e-12)

In [20]:
a = torch.rand(2,9,786).float()

In [10]:
a.shape

torch.Size([2, 9, 786])

In [18]:
from torch import nn
ln = nn.LayerNorm(786, eps=1e-12).double()

In [19]:
a1 = ln(a)

In [12]:
a1.shape

torch.Size([2, 9, 786])

In [6]:
import torch
a = torch.tensor([1,1,0])
b = torch.tensor([0.5,0,1])

In [8]:
torch.max(b-a, torch.tensor ([0.0]))

tensor([0., 0., 1.])

In [5]:
b-a

tensor([-1, -1,  1])

In [9]:
b = torch.full([2, 5], 1.5)

In [10]:
b

tensor([[1.5000, 1.5000, 1.5000, 1.5000, 1.5000],
        [1.5000, 1.5000, 1.5000, 1.5000, 1.5000]])

In [2]:
from datasets import load_dataset

train_dataset = load_dataset("philipphager/baidu-ultr_baidu-mlm-ctr",name="clicks",
                            split="train", # ["train", "test"]
                            cache_dir="~/.cache/huggingface",
                            )

/home/ggbhatt/anaconda3/envs/rank1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ggbhatt/anaconda3/envs/rank1/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for philipphager/baidu-ultr_baidu-mlm-ctr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/philipphager/baidu-ultr_baidu-mlm-ctr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
train_dataset

Dataset({
    features: ['query_id', 'query_md5', 'query', 'query_length', 'n', 'url_md5', 'text_md5', 'title', 'abstract', 'query_document_embedding', 'click', 'position', 'media_type', 'displayed_time', 'serp_height', 'slipoff_count_after_click', 'bm25', 'bm25_title', 'bm25_abstract', 'tf_idf', 'tf', 'idf', 'ql_jelinek_mercer_short', 'ql_jelinek_mercer_long', 'ql_dirichlet', 'document_length', 'title_length', 'abstract_length'],
    num_rows: 1779017
})

In [5]:
train_dataset['bm25'][0]

[37.58316421508789,
 36.20572280883789,
 56.07870864868164,
 70.85820007324219,
 52.38069152832031,
 47.444210052490234,
 51.77769088745117,
 41.325138092041016]

In [6]:
train_dataset['tf_idf'][0]

[25.439960479736328,
 20.177860260009766,
 41.639854431152344,
 35.50246810913086,
 38.605186462402344,
 25.439960479736328,
 39.025325775146484,
 22.449565887451172]

In [1]:
import torch
from src.bert import BertModel, BertReward
from transformers.models.bert.configuration_bert import BertConfig

config = BertConfig()

config.vocab = 100
config.num_labels = 1 # regression output
config.problem_type = "classification"
config.max_position_embeddings = 50

# if args.use_doc_feat:
# 	# self.config.doc_feat_len=0
# 	self.config.hidden_size+=12

reward_model = BertReward(config)

mname = "dfeat_un"
checkpoint = torch.load('../outputs/'+mname+'/checkpoint35.pth', map_location='cpu')

from datasets import load_dataset

train_dataset = load_dataset("philipphager/baidu-ultr_baidu-mlm-ctr",name="clicks",
                            split="train", # ["train", "test"]
                            cache_dir="~/.cache/huggingface",
                            )

/home/ggbhatt/anaconda3/envs/rank1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/ggbhatt/anaconda3/envs/rank1/lib/python3.9/site-packages/jax/_src/xla_bridge.py", line 430, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/ggbhatt/anaconda3/envs/rank1/lib/python3.9/site-packages/jax_plugins/xla_cuda12/__init__.py", line 78, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'
/home/ggbhatt/anaconda3/envs/rank1/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for philipphager/baidu-ultr_baidu-mlm-ctr 

In [3]:
from src.data import collate_fn
from torch.utils.data import DataLoader

train_dataset = load_dataset("philipphager/baidu-ultr_baidu-mlm-ctr",name="clicks",
                        split="train", # ["train", "test"]
                        cache_dir="~/.cache/huggingface",
                        )

test_dataset = load_dataset("philipphager/baidu-ultr_baidu-mlm-ctr",name="clicks",
                        split="test", # ["train", "test"]
                        cache_dir="~/.cache/huggingface",
                        )

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=64,
                                num_workers=2, pin_memory=True)
    
test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=64,
                                num_workers=2)

In [4]:
for batch in iter(train_dataloader):
    feat = batch['query_document_embedding']
    pos_idx = batch['position']
    out = reward_model(inputs_embeds=feat, position_ids=pos_idx)
    bre

/local/home/ggbhatt/workspace/cf_ranking/cf_rank/src/data.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400412039/work/torch/csrc/utils/tensor_new.cpp:261.)
  column: torch.tensor(features, dtype=COLUMNS[column]["dtype"])
/local/home/ggbhatt/workspace/cf_ranking/cf_rank/src/data.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400412039/work/torch/csrc/utils/tensor_new.cpp:261.)
  column: torch.tensor(features, dtype=COLUMNS[column]["dtype"])


: 

In [1]:
feat

NameError: name 'feat' is not defined

In [7]:
a['']

dict_keys(['n', 'query_document_embedding', 'click', 'position', 'bm25', 'bm25_title', 'bm25_abstract', 'tf_idf', 'tf', 'idf', 'ql_jelinek_mercer_short', 'ql_jelinek_mercer_long', 'ql_dirichlet', 'document_length', 'title_length', 'abstract_length', 'mask'])

In [2]:
import torch
import os

In [4]:
torch.save(torch.tensor([1,2,3]),os.path.join('__pycache__','def'))

In [1]:
14*7

98